# Ingest massive amounts of data to a Vector DB (Amazon OpenSearch)
**_Use of Amazon OpenSearch as a vector database for storing embeddings_**

This notebook works well with the `Data Science 2.0` kernel on a SageMaker Studio `ml.t3.xlarge` instance.

Here is a list of packages that are used in this notebook.
```
!pip list | grep -E -w "sagemaker|ipywidgets|langchain|opensearch-py|faiss-cpu|numpy|sh"
----------------------------------------------------------------------------------------
faiss-cpu                            1.7.4
ipywidgets                           7.6.5
langchain                            0.1.0
langchain-community                  0.0.20
langchain-core                       0.1.23
numpy                                1.26.2
opensearch-py                        2.2.0
sagemaker                            2.199.0
sagemaker-studio-image-build         0.6.0
sh                                   2.0.4
```

## Step 1: Setup
Install the required packages.

In [ ]:
!pip install -U langchain==0.0.350 opensearch-py==2.2.0 faiss_cpu==1.7.4 sh==2.0.4 sagemaker-studio-image-build==0.6.0 --quiet

In [ ]:
!pip list | grep -E -w "sagemaker|ipywidgets|langchain|opensearch-py|faiss-cpu|numpy|sh"

## Step 2: Download the data from the web and upload to S3

In this step we use `wget` to crawl a Python documentation style website data. All files other than `html`, `txt` and `md` are removed. **This data download would take a few minutes**.

In [ ]:
WEBSITE = "https://sagemaker.readthedocs.io/en/stable/"
DOMAIN = "sagemaker.readthedocs.io"
DATA_DIR = "docs"

In [ ]:
!python ./scripts/get_data.py --website {WEBSITE} --domain {DOMAIN} --output-dir {DATA_DIR}

In [ ]:
import boto3
import sagemaker

sagemaker_session = sagemaker.session.Session()
aws_region = boto3.Session().region_name
bucket = sagemaker_session.default_bucket()

In [ ]:
CREATE_OS_INDEX_HINT_FILE = "_create_index_hint"
app_name = 'llm-app-rag'

In [ ]:
# create a dummy file called _create_index to provide a hint for opensearch index creation
# this is needed for Sagemaker Processing Job when there are multiple instance nodes
# all running the same code for data ingestion but only one node needs to create the index
!touch {DATA_DIR}/{CREATE_OS_INDEX_HINT_FILE}

# upload this data to S3, to be used when we run the Sagemaker Processing Job
!aws s3 cp --recursive {DATA_DIR}/ s3://{bucket}/{app_name}/{DOMAIN}

## Step 3: Load data into OpenSearch

We now have a working script that is able to ingest data into an OpenSearch index. But for this to work for massive amounts of data we need to scale up the processing by running this code in a distributed fashion. We will do this using Sagemkaer Processing Job. This involves the following steps:

1. Create a custom container in which we will install the `langchain` and `opensearch-py` packges and then upload this container image to Amazon Elastic Container Registry (ECR).
2. Use the Sagemaker `ScriptProcessor` class to create a Sagemaker Processing job that will run on multiple nodes.
    - The data files available in S3 are automatically distributed across in the Sagemaker Processing Job instances by setting `s3_data_distribution_type='ShardedByS3Key'` as part of the `ProcessingInput` provided to the processing job.
    - Each node processes a subset of the files and this brings down the overall time required to ingest the data into Opensearch.
    - Each node also uses Python `multiprocessing` to internally also parallelize the file processing. Thus, **there are two levels of parallelization happening, one at the cluster level where individual nodes are distributing the work (files) amongst themselves and another at the node level where the files in a node are also split between multiple processes running on the node**.

### Create custom container

We will now create a container locally and push the container image to ECR. **The container creation process takes about 1 minute**.

1. The container include all the Python packages we need i.e. `langchain`, `opensearch-py`, `sagemaker` and `beautifulsoup4`.
1. The container also includes the `credentials.py` script for retrieving credentials from Secrets Manager and `sm_helper.py` for helping to create SageMaker endpoint classes that langchain uses.

In [ ]:
DOCKER_IMAGE = "load-data-opensearch-custom"
DOCKER_IMAGE_TAG = "latest"

In [ ]:
!cd ./container && sm-docker build . --repository {DOCKER_IMAGE}:{DOCKER_IMAGE_TAG}

### Create and run the Sagemaker Processing Job

Now we will run the Sagemaker Processing Job to ingest the data into OpenSearch.

In [ ]:
import os
import sys
import time
import json
import logging

import numpy as np
from typing import List

logger = logging.getLogger()
logging.basicConfig(format='%(asctime)s,%(module)s,%(processName)s,%(levelname)s,%(message)s', level=logging.INFO, stream=sys.stderr)

In [ ]:
def get_cfn_outputs(stackname: str, region_name: str) -> List:
    cfn = boto3.client('cloudformation', region_name=region_name)
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

In [ ]:
CFN_STACK_NAME = 'EmbeddingEndpointStack'

cfn_stack_outputs = get_cfn_outputs(CFN_STACK_NAME, aws_region)
embeddings_model_endpoint_name = cfn_stack_outputs['EmbeddingEndpointName']

In [ ]:
CFN_STACK_NAME = "RAGOpenSearchStack"
cfn_stack_outputs = get_cfn_outputs(CFN_STACK_NAME, aws_region)

opensearch_domain_endpoint = cfn_stack_outputs['OpenSearchDomainEndpoint']
opensearch_secretid = cfn_stack_outputs['OpenSearchSecret']
opensearch_index = 'llm_rag_embeddings'

In [ ]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
aws_role = sagemaker_session.get_caller_identity_arn()

In [ ]:
CHUNK_SIZE_FOR_DOC_SPLIT = 600
CHUNK_OVERLAP_FOR_DOC_SPLIT = 20

In [ ]:
from sagemaker.processing import (
    ProcessingInput,
    ScriptProcessor,
    FrameworkProcessor
)

# setup the parameters for the job
base_job_name = f"{app_name}-job"
tags = [{"Key": "data", "Value": "embeddings-for-llm-apps"}]

# use the custom container we just created
image_uri = f"{account_id}.dkr.ecr.{aws_region}.amazonaws.com/{DOCKER_IMAGE}:{DOCKER_IMAGE_TAG}"

# instance type and count determined via trial and error: how much overall processing time
# and what compute cost works best for your use-case
instance_type = "ml.m5.xlarge"
instance_count = 3
logger.info(f"base_job_name={base_job_name}, tags={tags}, image_uri={image_uri}, instance_type={instance_type}, instance_count={instance_count}")

# setup the ScriptProcessor with the above parameters
processor = ScriptProcessor(base_job_name=base_job_name,
                            image_uri=image_uri,
                            role=aws_role,
                            instance_type=instance_type,
                            instance_count=instance_count,
                            command=["python3"],
                            tags=tags)

# setup input from S3, note the ShardedByS3Key, this ensures that
# each instance gets a random and equal subset of the files in S3.
inputs = [ProcessingInput(source=f"s3://{bucket}/{app_name}/{DOMAIN}",
                          destination='/opt/ml/processing/input_data',
                          s3_data_distribution_type='ShardedByS3Key',
                          s3_data_type='S3Prefix')]


logger.info(f"creating an opensearch index with name={opensearch_index}")

# ready to run the processing job
st = time.time()
processor.run(code="container/load_data_into_opensearch.py",
              inputs=inputs,
              outputs=[],
              arguments=["--opensearch-cluster-domain", opensearch_domain_endpoint,
                         "--opensearch-secretid", opensearch_secretid,
                         "--opensearch-index-name", opensearch_index,
                         "--aws-region", aws_region,
                         "--embeddings-model-endpoint-name", embeddings_model_endpoint_name,
                         "--chunk-size-for-doc-split", str(CHUNK_SIZE_FOR_DOC_SPLIT),
                         "--chunk-overlap-for-doc-split", str(CHUNK_OVERLAP_FOR_DOC_SPLIT),
                         "--input-data-dir", "/opt/ml/processing/input_data",
                         "--create-index-hint-file", CREATE_OS_INDEX_HINT_FILE,
                         "--process-count", "2"])

time_taken = time.time() - st
logger.info(f"processing job completed, total time taken={time_taken}s")

preprocessing_job_description = processor.jobs[-1].describe()
logger.info(preprocessing_job_description)

## Step 4: Do a similarity search for for user input to documents (embeddings) in OpenSearch

In [ ]:
from container.credentials import get_credentials
from langchain.vectorstores import OpenSearchVectorSearch
from container.sm_helper import create_sagemaker_embeddings_from_js_model

creds = get_credentials(opensearch_secretid, aws_region)
http_auth = (creds['username'], creds['password'])

docsearch = OpenSearchVectorSearch(index_name=opensearch_index,
                                   embedding_function=create_sagemaker_embeddings_from_js_model(embeddings_model_endpoint_name,
                                                                                                aws_region),
                                   opensearch_url=opensearch_domain_endpoint,
                                   http_auth=http_auth)

q = "Which XGBoost versions does SageMaker support?"
docs = docsearch.similarity_search(q, k=3) #, search_type="script_scoring", space_type="cosinesimil"
for doc in docs:
    logger.info("----------")
    logger.info(f"content=\"{doc.page_content}\",\nmetadata=\"{doc.metadata}\"")

## Cleanup

To avoid incurring future charges, delete the resources. You can do this by deleting the CloudFormation template used to create the IAM role and SageMaker notebook.

---

## Conclusion
In this notebook we were able to see how to use LLMs deployed on a SageMaker Endpoint to generate embeddings and then ingest those embeddings into OpenSearch and finally do a similarity search for user input to the documents (embeddings) stored in OpenSearch. We used langchain as an abstraction layer to talk to both the SageMaker Endpoint as well as OpenSearch.

---

## Appendix

In [ ]:
from container.sm_helper import create_sagemaker_embeddings_from_js_model

CFN_STACK_NAME = 'EmbeddingEndpointStack'
cfn_stack_outputs = get_cfn_outputs(CFN_STACK_NAME, aws_region)
embeddings_model_endpoint_name = cfn_stack_outputs['EmbeddingEndpointName']

embeddings = create_sagemaker_embeddings_from_js_model(embeddings_model_endpoint_name, aws_region)

text = "This is a sample query."
query_result = embeddings.embed_query(text)

print(query_result)
print(f"length: {len(query_result)}")

## References

  * [Build a powerful question answering bot with Amazon SageMaker, Amazon OpenSearch Service, Streamlit, and LangChain](https://aws.amazon.com/blogs/machine-learning/build-a-powerful-question-answering-bot-with-amazon-sagemaker-amazon-opensearch-service-streamlit-and-langchain/)
  * [Using the Amazon SageMaker Studio Image Build CLI to build container images from your Studio notebooks](https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/)
  * [LangChain](https://python.langchain.com/docs/get_started/introduction.html) - A framework for developing applications powered by language models.